# Analisis de sentimientos Apli- [Large Movie Review Dataset v1.0](http://ai.stanford.edu/~amaas/data/sentiment/)

## Iván Vázquez Martínez

Se usó un script para extraer los datos y crear 2 archivos separados de train y test con todas las reviews:

#Script para extraer los datos descargados de http://ai.stanford.edu/~amaas/data/sentiment/ y crear 2 archivos con todos los datos en train y test\

#!/bin/bash

\# unzip y unpack el archivo tar:\
gunzip -c aclImdb_v1.tar.gz | tar xopf -

cd aclImdb

#creamos directorio donde estarán los archivos de train y test\
mkdir movie_data

\# Creamos los archivos donde estarán todos los datos de train y test, el orden es importante ya que las primeras 12.5k filas tendrán label positiva y las siguientes negativas\
\# full_train.txt, full_test.txt:\
for split in train test;\
do

  for sentiment in pos neg;\
  do 
    
    for file in $split/$sentiment/*; 
    do
              cat $file >> movie_data/full_${split}.txt; 
              echo >> movie_data/full_${split}.txt; 

	    
    done;
  done;\
done;

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Importamos librerias</h1></center>


In [1]:
import numpy as np
import pandas as pd
import os
import re
import glob
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Cargamos los datos</h1></center>


In [2]:
# train_test = [r"\train",r"\test"]
# pos_neg = [r"\pos",r"\neg"]

# all_files = []
# for i in range(len(train_test)):
#     for j in range(len(pos_neg)):
#         rel_path = train_test[i] + pos_neg[j]
#         os.chdir(r"C:\Users\sephc\Python\aclImdb"+ rel_path)
#         extension = 'txt'
#         files = [i for i in glob.glob('*.{}'.format(extension))]
#         all_files = all_files + files
# len(all_files)
# # Los primeros 25000 archivos son train de los cuales los primeros 12500 son pos (1) y los siguientes son neg (0)
# # luego los siguientes 25000 archivos son test de los cuales los primeros 12500 son pos (1) y los siguientes son neg (0).

# full_trainp = all_files[:25000]
# full_testp = all_files[25000:]

# len(full_trainp)

In [3]:
reviews_train = []
for line in open('./movie_data/full_train.txt', 'r', encoding="utf8"):
    
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('./movie_data/full_test.txt', 'r',encoding="utf8"):
    
    reviews_test.append(line.strip())

In [4]:
reviews_train[0]

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Preprocesamiento</h1></center>


Creamos la función preprocess_reviews() para reemplazar caracteres innecesarios:

In [5]:
#patterns
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

#funcion
def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    return reviews

#reviews limpias:
reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [6]:
reviews_train_clean[0]

'bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my  years in the teaching profession lead me to believe that bromwell highs satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled  at  high a classic line inspector im here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isnt'

In [7]:
len(reviews_train_clean)

25000

In [8]:
len(reviews_test_clean)

25000

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Vectorizamos</h1></center>


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Modelo y Accuracy</h1></center>


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

target = [1 if i < 12500 else 0 for i in range(25000)]

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr = LogisticRegression(C=c, max_iter = 1000)
    lr.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy con C=0.01: 0.87904
Accuracy con C=0.05: 0.88752
Accuracy con C=0.25: 0.88608
Accuracy con C=0.5: 0.88224
Accuracy con C=1: 0.87984


Con esto concluimos que el coeficiente de regularización óptimo es de C = 0.05:

In [11]:
modelo_final = LogisticRegression(C=0.05)
modelo_final.fit(X, target)
print ("Accuracy: %s"
       % accuracy_score(target, modelo_final.predict(X_test)))

Accuracy: 0.88144


In [12]:
modelo_final.coef_[0]

array([-0.00823153,  0.04268004,  0.04294586, ...,  0.00025657,
       -0.02673009, -0.00672057])

In [13]:
modelo_final.classes_

array([0, 1])

In [14]:
modelo_final.intercept_

array([0.28491058])

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">GridSearchCV</h1></center>

In [15]:
# from sklearn import svm
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression

# model_params = {
#     'svm': {
#         'model': svm.SVC(gamma='auto'),
#         'params' : {
#             'C': [1,5,10],
#             'kernel': ['rbf','linear']
#         }  
#     },
#     'random_forest': {
#         'model': RandomForestClassifier(),
#         'params' : {
#             'n_estimators': [1,5,10]
#         }
#     }
# }

In [16]:
# from sklearn.model_selection import GridSearchCV
# scores = []

#for model_name, mp in model_params.items():
#     clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
#     clf.fit(X, target)
#     scores.append({
#         'model': model_name,
#         'best_score': clf.best_score_,
#         'best_params': clf.best_params_
#     })
    
#df_results = pd.DataFrame(scores,columns=['model','best_score','best_params'])
#df_results

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Curva ROC</h1></center>


In [17]:
# from sklearn.metrics import roc_curve, roc_auc_score
# y_pred_proba= modelo_final.predict_proba(X_val)
# fpr, tpr, thresholds = roc_curve(y_val,y_pred_proba[:,1])

# plt.plot(fpr,tpr)
# plt.plot([0,1],[0,1],linestyle='--')
# plt.xlim([0.0,1.0])
# plt.ylim([0.0,1.0])
# plt.xlabel('1-specificity')
# plt.ylabel('sensitivity')
# plt.show()

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Mejores 5 y peores palabras</h1></center>


In [18]:
feature_to_coef = {word: coef for word, coef in zip(cv.get_feature_names(), modelo_final.coef_[0])}

for best_positive in sorted(feature_to_coef.items(), key=lambda x: x[1], reverse=True)[:5]:
    print (best_positive)

('excellent', 0.9283544280664209)
('perfect', 0.7944278054002185)
('great', 0.6745552418750913)
('amazing', 0.6164834634404539)
('superb', 0.6055919567375537)


In [19]:
for best_negative in sorted(feature_to_coef.items(), key=lambda x: x[1])[:5]:
    print (best_negative)

('worst', -1.367989784676191)
('waste', -1.1688808874334595)
('awful', -1.0273337636998021)
('poorly', -0.8748022397941021)
('boring', -0.8591221323933731)


### Quitamos las Stopwords

In [20]:
#import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

english_stop_words = stopwords.words('english')

def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

no_stop_words_train = remove_stop_words(reviews_train_clean)
no_stop_words_test = remove_stop_words(reviews_test_clean)

cv = CountVectorizer(binary=True)
cv.fit(no_stop_words_train)
X = cv.transform(no_stop_words_train)
X_test = cv.transform(no_stop_words_test)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c, max_iter = 1000)
    lr.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy con C=0.01: 0.87104
Accuracy con C=0.05: 0.88272
Accuracy con C=0.25: 0.8816
Accuracy con C=0.5: 0.88048
Accuracy con C=1: 0.8792


Con C = 0.05 se obtiene la mejor accuracy

## Radicalización (stemming)

In [21]:
def get_stemmed_text(corpus):
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

stemmed_reviews_train = get_stemmed_text(reviews_train_clean)
stemmed_reviews_test = get_stemmed_text(reviews_test_clean)

cv = CountVectorizer(binary=True)
cv.fit(stemmed_reviews_train)
X = cv.transform(stemmed_reviews_train)
X_test = cv.transform(stemmed_reviews_test)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
final_stemmed = LogisticRegression(C=0.05, max_iter = 5000)
final_stemmed.fit(X, target)
print ("Accuracy: %s" 
       % accuracy_score(target, final_stemmed.predict(X_test)))

Accuracy con C=0.01: 0.87168
Accuracy con C=0.05: 0.87536


C:\Users\sephc\.conda\envs\python_env_38\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy con C=0.25: 0.87408


C:\Users\sephc\.conda\envs\python_env_38\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy con C=0.5: 0.87072


C:\Users\sephc\.conda\envs\python_env_38\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy con C=1: 0.86784
Accuracy: 0.8772


## Lematización

In [22]:
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def get_lemmatized_text(corpus):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

lemmatized_reviews_train = get_lemmatized_text(reviews_train_clean)
lemmatized_reviews_test = get_lemmatized_text(reviews_test_clean)

cv = CountVectorizer(binary=True)
cv.fit(lemmatized_reviews_train)
X = cv.transform(lemmatized_reviews_train)
X_test = cv.transform(lemmatized_reviews_test)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c,max_iter = 2000)
    lr.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
final_lemmatized = LogisticRegression(C=0.25)
final_lemmatized.fit(X, target)
print ("Accuracy: %s" 
       % accuracy_score(target, final_lemmatized.predict(X_test)))

Accuracy con C=0.01: 0.87856
Accuracy con C=0.05: 0.8872
Accuracy con C=0.25: 0.88544
Accuracy con C=0.5: 0.88048
Accuracy con C=1: 0.87664
Accuracy: 0.87364


C:\Users\sephc\.conda\envs\python_env_38\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## n-grams

In [23]:
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c, max_iter = 5000)
    lr.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
final_ngram = LogisticRegression(C=0.5, max_iter = 5000)
final_ngram.fit(X, target)
print ("Accuracy: %s" 
       % accuracy_score(target, final_ngram.predict(X_test)))

Accuracy con C=0.01: 0.88224
Accuracy con C=0.05: 0.88992
Accuracy con C=0.25: 0.89056
Accuracy con C=0.5: 0.89056
Accuracy con C=1: 0.89104
Accuracy: 0.89872


### TF-IDF

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(reviews_train_clean)
X = tfidf_vectorizer.transform(reviews_train_clean)
X_test = tfidf_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
final_tfidf = LogisticRegression(C=1, max_iter = 2000)
final_tfidf.fit(X, target)
print ("Accuracy: %s" 
       % accuracy_score(target, final_tfidf.predict(X_test)))

Accuracy con C=0.01: 0.79552
Accuracy con C=0.05: 0.8264
Accuracy con C=0.25: 0.86624
Accuracy con C=0.5: 0.87616
Accuracy con C=1: 0.88544
Accuracy: 0.88248


### Linear SVC

In [25]:
from sklearn.svm import LinearSVC

ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c, max_iter = 2000)
    svm.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val))))
    

final_svm_ngram = LinearSVC(C=0.01,max_iter = 2000)
final_svm_ngram.fit(X, target)
print ("Accuracy: %s" 
       % accuracy_score(target, final_svm_ngram.predict(X_test)))

Accuracy con C=0.01: 0.88896
Accuracy con C=0.05: 0.88832
Accuracy con C=0.25: 0.88592
Accuracy con C=0.5: 0.88592
Accuracy con C=1: 0.88576
Accuracy: 0.8976


### LinearSVC sin Stopwords

In [26]:
stop_words = ['in', 'of', 'at', 'a', 'the']
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 3), stop_words=stop_words)
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

for c in [0.001, 0.005, 0.01, 0.05, 0.1]:
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy con C=%s: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val))))
    
final = LinearSVC(C=0.01)
final.fit(X, target)
print ("Accuracy: %s" 
       % accuracy_score(target, final.predict(X_test)))

Accuracy con C=0.001: 0.88704
Accuracy con C=0.005: 0.89376
Accuracy con C=0.01: 0.89312
Accuracy con C=0.05: 0.8944
Accuracy con C=0.1: 0.89472
Accuracy: 0.9


### XGBoost

In [27]:
# from xgboost import XGBRegressor
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import train_test_split

# stop_words = ['in', 'of', 'at', 'a', 'the']
# ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 3), stop_words=stop_words)
# ngram_vectorizer.fit(reviews_train_clean)
# X = ngram_vectorizer.transform(reviews_train_clean)
# X_test = ngram_vectorizer.transform(reviews_test_clean)

# X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.75)

# estimators = [100,300,500,800,1000]

# xgboost = XGBRegressor(n_estimators =100 , learning_rate = 0.05)
# xgboost.fit(X_train, y_train,
#             early_stopping_rounds = 5,
#             eval_set =[(X_val,y_val)],
#             verbose = False)
# print ("Accuracy for n=%s: %s" 
#         % (100, accuracy_score(y_val, xgboost.predict(X_val))))

In [34]:
X_train.get_shape()

(18750, 5398364)

In [38]:
len(y_train)

18750

In [41]:
#!pip install keras

In [43]:
#!pip install tensorflow

  Using cached wrapt-1.12.1.tar.gz (27 kB)
  Created wheel for wrapt: filename=wrapt-1.12.1-py3-none-any.whl size=19558 sha256=84b4642228e1e8058ddd6d115cf62ce46efc0fa8490edf82644ca82b9924676a
  Stored in directory: c:\users\sephc\appdata\local\pip\cache\wheels\5f\fd\9e\b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=486b361e8a04828136460ad792788484bcadea7c275224e7a83ca022fb3b28bf
  Stored in directory: c:\users\sephc\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built wrapt termcolor
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2


### Neural Network

In [46]:
from keras import models
from keras import layers

model = models.Sequential()
# Input - Layer
model.add(layers.Dense(50, activation = "relu", input_shape=(18750, )))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

# Compilamos el modelo
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

results = model.fit(
 X_train, y_train,
 epochs= 2,
 batch_size = 500,
 validation_data = (X_val, y_val))

print("Test-Accuracy:", np.mean(results.history["val_acc"]))

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

<center><h1 style = "background-image: linear-gradient(90deg, black, blue);border:0;color:white">Creación de Dataframe</h1></center>


In [28]:
reviews_test_series = pd.Series(reviews_test_clean)
reviews_train_series = pd.Series(reviews_train_clean)

target_series = pd.Series(target)

frame_test = { 'Reviews': reviews_test_series, 'Sentiment': target_series }
frame_train = { 'Reviews': reviews_train_series, 'Sentiment': target_series }

df_test = pd.DataFrame(frame_test)
df_train = pd.DataFrame(frame_train)

df_train[12498:12502]

,Reviews,Sentiment
12498,a christmas together actually came before my t...,1
12499,working class romantic drama from director mar...,1
12500,story of a man who has unnatural feelings for ...,0
12501,airport starts as a brand new luxury plane i...,0


In [29]:
target_series[12498:12502]

12498    1
12499    1
12500    0
12501    0
dtype: int64

In [30]:
df_train.describe()

,Sentiment
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000
